In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error
from google.cloud import bigquery
from google.oauth2 import service_account
from math import sqrt
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SA_FILEPATH = "/content/drive/MyDrive/Insight Estoque/service_account_unif.json"

credentials = service_account.Credentials.from_service_account_file(
    SA_FILEPATH, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [ ]:
project_name = 'carol-0483dfa91ba54cd4b97b.0483dfa91ba54cd4b97b1613f1171932'

### Compras

##### Períodos com previsões

In [ ]:
query = """
  SELECT tenantid, id, frequency, count(*) points
  FROM `{0}.deduplicated_demandgraph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_demandgraph` )  AND prediction
  group by tenantid, id, frequency
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df.loc[(df.frequency == 'M') ]['points'].unique()

<IntegerArray>
[3]
Length: 1, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'W') ]['points'].unique()

<IntegerArray>
[4]
Length: 1, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'D') ]['points'].unique()

<IntegerArray>
[7]
Length: 1, dtype: Int64

##### Períodos de histórico

In [ ]:
query = """
  SELECT tenantid, id, frequency, count(*) points
  FROM `{0}.deduplicated_demandgraph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_demandgraph` )  AND NOT prediction
  group by tenantid, id, frequency
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df.loc[(df.frequency == 'M') ]['points'].unique()

<IntegerArray>
[9]
Length: 1, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'W') ]['points'].unique()

<IntegerArray>
[13]
Length: 1, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'D') ]['points'].unique()

<IntegerArray>
[47]
Length: 1, dtype: Int64

##### Total de períodos

In [ ]:
query = """
  SELECT tenantid, id, frequency, count(*) points
  FROM `{0}.deduplicated_demandgraph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_demandgraph` )
  group by tenantid, id, frequency
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df.loc[(df.frequency == 'M') ]['points'].unique()

<IntegerArray>
[12]
Length: 1, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'W') ]['points'].unique()

<IntegerArray>
[17]
Length: 1, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'D') ]['points'].unique()

<IntegerArray>
[54]
Length: 1, dtype: Int64

##### Total de previsões

In [ ]:
query = """
  SELECT COUNT(DISTINCT(CONCAT(id, frequency))) total
  FROM `{0}.deduplicated_demandgraph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_demandgraph` )
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df

,total
0,12205


In [ ]:
query = """
  SELECT COUNT(DISTINCT(CONCAT(id, frequency))) total
  FROM `{0}.deduplicated_demandalerts`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_demandalerts` )
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df

,total
0,12205


In [ ]:
query = """
  SELECT COUNT(DISTINCT(CONCAT(serieId, data_frequency))) total
  FROM `{0}.deduplicated_stg_demand_forecasting_demand_forecasting`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_stg_demand_forecasting_demand_forecasting` )
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df

,total
0,12205


##### Períodos

In [ ]:
query = """
  SELECT initial_reference_date, frequency, prediction
  FROM `{0}.deduplicated_demandgraph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_demandgraph` )
  group by initial_reference_date, frequency, prediction
  order by initial_reference_date
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df.loc[df['frequency']=='M']

,initial_reference_date,frequency,prediction
0,2023-03-01 03:00:00,M,False
1,2023-04-01 03:00:00,M,False
2,2023-05-01 03:00:00,M,False
3,2023-06-01 03:00:00,M,False
4,2023-07-01 03:00:00,M,False
5,2023-08-01 03:00:00,M,False
6,2023-09-01 03:00:00,M,False
9,2023-10-01 03:00:00,M,False
16,2023-11-01 03:00:00,M,False
51,2023-12-01 03:00:00,M,True


In [ ]:
df.loc[df['frequency']=='W']

,initial_reference_date,frequency,prediction
7,2023-09-17 03:00:00,W,False
8,2023-09-24 03:00:00,W,False
10,2023-10-01 03:00:00,W,False
11,2023-10-08 03:00:00,W,False
12,2023-10-15 03:00:00,W,False
13,2023-10-22 03:00:00,W,False
14,2023-10-29 03:00:00,W,False
20,2023-11-05 03:00:00,W,False
29,2023-11-12 03:00:00,W,False
36,2023-11-19 03:00:00,W,False


In [ ]:
df.loc[df['frequency']=='D']

,initial_reference_date,frequency,prediction
15,2023-11-01 03:00:00,D,False
17,2023-11-02 03:00:00,D,False
18,2023-11-03 03:00:00,D,False
19,2023-11-04 03:00:00,D,False
21,2023-11-05 03:00:00,D,False
22,2023-11-06 03:00:00,D,False
23,2023-11-07 03:00:00,D,False
24,2023-11-08 03:00:00,D,False
25,2023-11-09 03:00:00,D,False
26,2023-11-10 03:00:00,D,False


### Estoque

##### Períodos com previsões

In [ ]:
query = """
  SELECT tenantid, id, frequency, count(*) points
  FROM `{0}.deduplicated_rupturegraph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_rupturegraph` )  AND prediction
  group by tenantid, id, frequency
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df.loc[(df.frequency == 'M') ]['points'].unique()

<IntegerArray>
[3]
Length: 1, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'W') ]['points'].unique()

<IntegerArray>
[4]
Length: 1, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'D') ]['points'].unique()

<IntegerArray>
[7]
Length: 1, dtype: Int64

##### Períodos de histórico

In [ ]:
query = """
  SELECT tenantid, id, frequency, count(*) points
  FROM `{0}.deduplicated_rupturegraph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_rupturegraph` )  AND NOT prediction
  group by tenantid, id, frequency
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df.loc[(df.frequency == 'M') ]['points'].unique()

<IntegerArray>
[9, 8, 7, 5, 6, 4, 2]
Length: 7, dtype: Int64

##### Total de períodos

In [ ]:
query = """
  SELECT tenantid, id, frequency, count(*) points
  FROM `{0}.deduplicated_rupturegraph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_rupturegraph` )
  group by tenantid, id, frequency
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df.loc[(df.frequency == 'M') ]['points'].unique()

<IntegerArray>
[3, 12, 11, 10, 8, 9, 7, 5]
Length: 8, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'W') ]['points'].unique()

<IntegerArray>
[4]
Length: 1, dtype: Int64

In [ ]:
df.loc[(df.frequency == 'D') ]['points'].unique()

<IntegerArray>
[7]
Length: 1, dtype: Int64

##### Total de previsões

In [ ]:
query = """
  SELECT COUNT(DISTINCT(CONCAT(id, frequency))) total
  FROM `{0}.deduplicated_rupturegraph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_rupturegraph` ) and prediction
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df

,total
0,11033


In [ ]:
query = """
  SELECT COUNT(DISTINCT(CONCAT(id, frequency))) total
  FROM `{0}.stg_demand_forecasting_rupture_graph`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.stg_demand_forecasting_rupture_graph` )
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df

,total
0,11030


In [ ]:
query = """
  SELECT COUNT(DISTINCT(CONCAT(id, frequency))) total
  FROM `{0}.deduplicated_rupturealert`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_rupturealert` )
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df

,total
0,10550


In [ ]:
query = """
  SELECT COUNT(DISTINCT(CONCAT(id, frequency))) total
  FROM `{0}.stg_demand_forecasting_rupture_alerts`
  WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.stg_demand_forecasting_rupture_alerts` )
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

In [ ]:
df

,total
0,10550


In [ ]:
query = """
  select count(*)
  from `{0}.deduplicated_stg_protheus_carol_cv3`
  where length(protheus_pk) <= 3
""".format(project_name)

In [ ]:
df = client.query(query).to_dataframe()

KeyboardInterrupt: ignored

In [ ]:
df

,f0_
0,917682993
